In [1]:
library(shiny)
library(shinythemes)  
library(dplyr)
library(ggplot2)
library(reshape2)
library(patternplot)
library(tidyr)
library(scales)
library(rtweet)
library(data.table)
library(twitteR)
library(maps)
library(maptools)
library(ggmap)
library(revgeo)



Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Attaching package: 'tidyr'

The following object is masked from 'package:reshape2':

    smiths


Attaching package: 'data.table'

The following objects are masked from 'package:reshape2':

    dcast, melt

The following objects are masked from 'package:dplyr':

    between, first, last


Attaching package: 'twitteR'

The following object is masked from 'package:rtweet':

    lookup_statuses

The following objects are masked from 'package:dplyr':

    id, location

Loading required package: sp
Checking rgeos availability: FALSE
 	Note: when rgeos is not available, polygon geometry 	computations in maptools depend on gpclib,
 	which has a restricted licence. It is disabled by default;
 	to enable gpclib, type gpclibPermit()
Google's Terms of Service: https://cloud.google.com/maps-platform/te

In [2]:
visualization <- function(x,npar=TRUE,print=TRUE) {    
#tweet_1 <- read.csv("MyTweets.csv",header=T)
library(usmap)
m <- us_map(regions = "states")
STATES <- data.frame("state" = unique(m$full))
STATES$state <- tolower(STATES$state)
sum(lengths(x$state))
#lengths(Data)
Data_temp <- data.frame(table(x$state))
Mi = min(Data_temp$Freq) 
Ma = max(Data_temp$Freq)
avg = Mi+Ma/2 
print(Ma/10)
print(Mi)
print(Ma)
Data_temp$state <- tolower(Data_temp$Var1)
Data_temp <- merge(x=Data_temp, y=STATES, by.x=c("state"), by.y=c("state"), all.y=TRUE,all.x=T)
Data_temp$FIPS <- fips(Data_temp$state)
Data_temp <- Data_temp[!(is.na(Data_temp$FIPS)),]
#fwrite(Data_temp,"myplot.csv")
library(usmap)
plt6 <-plot_usmap(data = Data_temp,values = "Freq",lines = "black") + 
       scale_fill_gradientn(colors =c("#00ff00","#40ff00","#80ff00",
                                      "#bfff00","#ffff00","#ffbf00",
                                      "#ff8000","#ff4000","#ff0000"),na.value= "#fcffef",
                            labels = c("Minimal", "Minimal","Minimal",
                                       "Low","Low",
                                       "moderate", "moderate", 
                                       "High","High"),
           breaks = seq(1,Ma,by=Ma/9) ,guide = guide_colourbar(barwidth =0.6,barheight =8))+
       labs(fill='ILI ACTIVITY LEVEL')+theme(legend.position = "right")#+ guides(fill=guide_legend(reverse=T))
       results <- list()
       results$first <- Data_temp
       results$second <-plt6
return(results)
}#c(5,10,15,20,25,30,35,40,45,51)
tweet_1 <- read.csv("MyTweets.csv",header=T)
plt1 <- visualization(tweet_1)
#plt1$first


[1] 52.2
[1] 1
[1] 522


In [ ]:
ui <- fluidPage(theme = shinytheme("lumen"),
  titlePanel("ILI ACTIVITY LEVEL"),
  sidebarLayout(
    sidebarPanel(width =3,

      # Select type of trend to plot
      selectInput(inputId = "type", label = strong("Select a data"),
                  choices = c("CDC_HEAT_MAP","TWEETS_HEAT_MAP","#FLU_HEATMAP","#INFLUENZA_HEATMAP"), 
                  selected = "CDC_HEAT_MAP"),
      selectInput(inputId = "type_1", label = strong("Select a data"),
                  choices = c("CDC_HEAT_MAP","TWEETS_HEAT_MAP","#FLU_HEATMAP","#INFLUENZA_HEATMAP"),
                  selected = "TWEETS_HEAT_MAP"),

      # Display only if the smoother is checked
      conditionalPanel(condition = "input.smoother == true",
                       sliderInput(inputId = "f", label = "Smoother span:",
                                   min = 0.01, max = 1, value = 0.67, step = 0.01,
                                   animate = animationOptions(interval = 100)),
                       HTML("Higher values give more smoothness.")
      )
    ),

    # Output: Description, lineplot, and reference
   #mainPanel(
   #  plotOutput(outputId = "lineplot", height = "300px"),
   #  textOutput(outputId = "desc"),
   #  tags$a(href = "https://www.cdc.gov/flu/weekly/index.htm", "Source: CDC.gov", target = "_blank")
   #)
    mainPanel(width = 9, 
              tabsetPanel( 
              tabPanel(title = 'Output Map', 
              plotOutput(outputId = "map_1"),
              plotOutput(outputId = "map_2")),
              tabPanel(title = 'Data Table_1', 
              dataTableOutput(outputId = 'table_1')),
              tabPanel(title = 'Data Table_2', 
              dataTableOutput(outputId = 'table_2'))),
              tags$a(href = "https://www.cdc.gov/flu/weekly/index.htm", "Source: CDC.gov", target = "_blank")
  )
))

# Define server function
server <- function(input, output)
    {
   #=================type1================
    output$map_1 <- renderPlot({
     if(input$type =="CDC_HEAT_MAP"){
       plt1$second
   } 

})
  
  output$table_1 <- renderDataTable({
   if(input$type =="CDC_HEAT_MAP"){
       plt1$first
   }
})
    #=================type2==============
        output$map_2 <- renderPlot({
     if(input$type_1 =="CDC_HEAT_MAP"){
       plt1$second
   } 

})
  
  output$table_2 <- renderDataTable({
   if(input$type_1 =="CDC_HEAT_MAP"){
       plt1$first
   }
})
   
}
# Create Shiny object
shinyApp(ui = ui, server = server)
runApp(launch.browser = TRUE)


Listening on http://127.0.0.1:5655
